In [1]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

In [2]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [3]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [4]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [5]:
xlsx_resumen

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGADO
0,3,2025-05-24,NaN,NaN,52.72
1,3,2025-05-26,1.065976,65.266759,14.99
2,3,2025-06-02,NaN,NaN,28.46
3,4,2025-05-24,NaN,NaN,75.16
4,4,2025-05-26,NaN,NaN,25.34
...,...,...,...,...,...
269,602,2025-06-02,2.041940,101.311447,256.24
270,609,2025-05-24,NaN,NaN,93.54
271,609,2025-05-26,2.818424,NaN,81.98
272,609,2025-05-29,1.466327,92.234535,NaN


In [6]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [7]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [8]:
list_df = []

In [9]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

In [10]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [11]:
df_combinado

,COD_COS,FECHA,AREA,ESTIMADO,ENTREGAS
0,3,2025-05-26,1.065976,65.266759,67.71
1,3,2025-06-02,0.000000,0.000000,28.46
2,4,2025-06-02,0.000000,0.000000,236.49
3,5,2025-05-24,6.401479,376.148855,374.85
4,5,2025-05-26,2.579450,138.270525,430.17
...,...,...,...,...,...
211,417,2025-06-02,9.586049,595.455976,577.54
212,420,2025-05-26,2.443217,184.915300,552.78
213,451,2025-06-02,1.804662,60.535800,42.36
214,468,2025-06-02,7.053169,498.875476,721.63


In [12]:
df_combinado.to_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')